In [1]:
import numpy as np
import re
import pandas as pd
import os
import codecs
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/' 

In [2]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim import utils
from stop_words import get_stop_words
import nltk
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import normalize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import pymorphy2
import pymystem3

D:\distr\anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:
df_neg = pd.read_csv('negative.csv', delimiter=';',header=None )
df_pos = pd.read_csv('positive.csv', delimiter=';',header=None )

In [4]:
df_neg.columns = ['id','tdate','tmane','ttext','ttype','trep','trtw','tfav','tstcount','tfol','tfrien','listcount']
df_pos.columns = ['id','tdate','tmane','ttext','ttype','trep','trtw','tfav','tstcount','tfol','tfrien','listcount']

In [5]:
df_neg.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [6]:
df_pos.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [7]:
df =pd.concat((df_pos, df_neg), axis=0)

In [8]:
y =df.ttype

In [9]:
del df['ttype']

In [10]:
df.drop(['id', 'tdate'], inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 0 to 111922
Data columns (total 11 columns):
id           226834 non-null int64
tdate        226834 non-null int64
tmane        226834 non-null object
ttext        226834 non-null object
trep         226834 non-null int64
trtw         226834 non-null int64
tfav         226834 non-null int64
tstcount     226834 non-null int64
tfol         226834 non-null int64
tfrien       226834 non-null int64
listcount    226834 non-null int64
dtypes: int64(9), object(2)
memory usage: 20.8+ MB


In [12]:
df.tmane.nunique()

139658

In [13]:
df.drop(['tmane'], inplace=True)

In [14]:
import matplotlib
import seaborn as sns

In [15]:
%config InlineBackend.figure_format = 'png' 
sns_plot = sns.pairplot(
    df);
sns_plot.savefig('pairplot.png')

In [143]:
class Morpher():
    '''
    activation - distance function betven labeldata and answer
    '''

    def __init__(self, model_rucopr,mystem, pymorphy ):
        self.model_rucopr=model_rucopr
        self.mystem = mystem
        self.pymorph = pymorphy
        self.count=0
    def normality_me(self,query):
        if (self.count%10000) ==0:
            print(self.count)
        self.count+=1
        penn_to_wn = { 
            'S': wn.NOUN, 
            'V': wn.VERB, 
            'ADV': wn.ADV, 
            'A': wn.ADJ}
        pym_to_rutag ={
            'S': 'NOUN',  
            'V': 'VERB',
            'ADV': 'ADV', 
            'A': 'ADJ'  }          
    
#        tags = [(morph.parse(word)[0].word, morph.parse(word)[0].tag.POS) for word in  word_tokenize(query)]
        
#         tmpinput = codecs.open('tmpinput', 'w', 'utf-8') 
#         tmpinput.write(query)
#         tmpinput.close()
#         tmpoutput = codecs.open('tmpoutput', 'w', 'utf-8')
#         tmpoutput.close()
#         !mystem -dnil -e utf-8 tmpinput tmpoutput
#         tmpoutput = codecs.open('tmpoutput', 'r', 'utf-8')
#         lines = tmpoutput.readlines()
#         tmpoutput.close()
#         words = self.mystem.lemmatize(query)


        words = [self.pymorph.parse(s)[0][2] for s in  word_tokenize(query)]
        return ' '.join(words)

        patternnotvalid = re.compile('.*\?.*')
        tags = [(tstres[11]['analysis'][0]['gr'].split(',')[0],tstres[11]['analysis'][0]['lex']) for line in lines if not bool(patternnotvalid.match(tstres[11]['analysis'][0]['lex'])) ]

                                            
        words=[]
        for tag in tags:
            wn_tag = penn_to_wn.get(tag[1], wn.NOUN)
            word = morph.normal_forms(tag[0])[0]
            words.append((WordNetLemmatizer().lemmatize(word,wn_tag).lower() + "_" + pym_to_rutag.get(tag[1], 'NOUN')))
        return " ".join(words)
    def text_to_vec(self, text):
        if (len(text)==0) | (text is None):
            return (np.zeros((1,300)), '', '' , '')
        if 1==1:
            pattern= re.compile('[^А-Яа-я\s]+')
            pattern_spaces = re.compile('\s{1,}')
            text = pattern_spaces.sub(' ', pattern.sub(' ', text))
            text = self.normality_me(text)
            
            words = text.split(' ')
            words_err = []
            words_true = []
            sumvec = np.zeros((300,))
            
            for word in words:
                try:
                    vec = self.model_rucopr.word_vec(word)
                    sumvec = sumvec + vec
                    words_true.append(word)
                except:
                    words_err.append(word)
        return(normalize(sumvec.reshape(1, -1)), words, words_true, words_err)

In [50]:
model_rucopr = KeyedVectors.load_word2vec_format('D:\home\datasets\\ruwikiruscorpora_0_300_20.bin', binary=True)

In [127]:
pymorph = pymorphy2.MorphAnalyzer()
mystem = pymystem3.Mystem()

In [138]:
pymorph.parse('мыла')[0][2]

'мыло'

In [254]:
mystem.lemmatize('Мама мыла раму') ## намного лучше но оооочень долго, по прогнозам больше 20 часов

['мама', ' ', 'мыть', ' ', 'рама', '\n']

In [149]:
mymorph = Morpher(model_rucopr,mystem,pymorph )

In [145]:
%%time
testlemotise=[mymorph.normality_me(s) for s in df.ttext.head(10)]

0
Wall time: 72 ms


In [146]:
testlemotise

['@ first_timee хоть я и школотый , но поверь , у мы то же самый : d общество профилировать предмет тип )',
 'да , всё-таки он немного похожий на он . но мой мальчик весь равно хороший : d',
 'rt @ katiacheh : ну ты идиотка ) я испугаться за ты ! ! !',
 "rt @ digger2912 : `` кто то в угол сидеть и погибать от голод , а мы ещё 2 порция взять , хотя уже и так жрать не хотеть '' : dd http : //t.co/gqg6iue2…",
 '@ irina_dyshkant вот что значит страшилка : d но блин , посмотреть весь часть , у ты создаться ощущение , что автор курить что-то : d',
 'ну любить или нет ? — я не знать кто ты бля : d http : //t.co/brf9eng1u6',
 'rt @ spoonlamer : ох,900 : d ну это конечно же @ twinkleayo . чтобы у нея быть много друг , ведь она такой мимими & lt ; 3',
 'rt @ veregijytaqo : у ты есть ухажёр ? нет - мой ухо не кто не жрать : d',
 'поприветствовать мой новое читатель @ alexey1789 ; )',
 'теперь у я есть частичка сидней : ) # sydney # sydneyoperahouse http : //t.co/agnto3czei']

In [150]:
%%time
df['ttext_norm'] = [mymorph.normality_me(s) for s in df.ttext ]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [151]:
import pickle

In [154]:
file = open('df', 'wb')
pickle.dump(df,file)
file.close()

In [155]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [156]:
y_tr, y_val, x_tr,  x_val = train_test_split(y, df, test_size=0.2, random_state=42 )

In [163]:
text = x_tr.ttext_norm

In [198]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [159]:
from nltk.corpus import stopwords

In [160]:
stopwords.words('russian')[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [262]:
countvec = CountVectorizer(min_df= 5, stop_words= stopwords.words('russian'),
                            preprocessor= lambda s: re.sub('http',' ', s) 
                          )

In [263]:
countvec2 = CountVectorizer(min_df= 5, stop_words= stopwords.words('russian'),
                            preprocessor= lambda s: re.sub('http',' ', s) 
                          )

In [264]:
tfidf = TfidfVectorizer(min_df= 5, stop_words= stopwords.words('russian'))

In [265]:
tfidf2 = TfidfVectorizer(min_df= 5, stop_words= stopwords.words('russian'))

In [266]:
%%time
data_cv      = countvec.fit_transform(x_tr.ttext)
data_cv_norm = countvec2.fit_transform(x_tr.ttext_norm)

Wall time: 10.4 s


In [267]:
%%time
data_tf      = tfidf.fit_transform(x_tr.ttext)
data_tf_norm = tfidf2.fit_transform(x_tr.ttext_norm)

Wall time: 10.1 s


In [268]:
ss_cv = StandardScaler(with_mean=False)
ss_cv_norm = StandardScaler(with_mean=False)
ss_tf = StandardScaler(with_mean=False)
ss_tf_norm = StandardScaler(with_mean=False)

In [269]:
data_cv_ss=ss_cv.fit_transform(data_cv)
data_cv_norm_ss=ss_cv_norm.fit_transform(data_cv_norm)
data_tf_ss = ss_tf.fit_transform(data_tf)
data_tf_norm_ss = ss_tf_norm.fit_transform(data_tf_norm)

D:\distr\anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [271]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [300]:
logreg = LogisticRegression(C=0.0001)

In [273]:
from sklearn.model_selection import cross_val_score

In [283]:
res={}

In [284]:
res['cv'] = np.mean(cross_val_score(logreg, data_cv, y_tr, cv = 5, scoring='accuracy'))

In [285]:
res['cv_ss'] = np.mean(cross_val_score(logreg, data_cv_ss, y_tr, cv = 5, scoring='accuracy'))

In [286]:
res['cv_norm_ss'] = np.mean(cross_val_score(logreg, data_cv_norm_ss, y_tr, cv = 5, scoring='accuracy'))

In [287]:
res['tf'] = np.mean(cross_val_score(logreg, data_tf, y_tr, cv = 5, scoring='accuracy'))

In [288]:
res['tf_ss'] = np.mean(cross_val_score(logreg, data_tf_ss, y_tr, cv = 5, scoring='accuracy'))

In [289]:
res['tf_norm_ss'] = np.mean(cross_val_score(logreg, data_tf_norm_ss, y_tr, cv = 5, scoring='accuracy'))

In [290]:
res

{'cv': 0.59974540271688526,
 'cv_norm_ss': 0.73393508475778779,
 'cv_ss': 0.73205595166511306,
 'tf': 0.50830729873399183,
 'tf_norm_ss': 0.72964230604404479,
 'tf_ss': 0.72960370013696774}

### как мы видим нормализация помогла совсем чуть чуть, больше буста дал - StandartScaler 
### tfidf чуть хуже CountVec в данном случае, думаю потому, что много опечаток, а тексты короткие

In [331]:
logreg.fit(data_cv_norm_ss, y_tr)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [332]:
logreg.coef_[0,np.argsort(logreg2.coef_)]

array([[ 0.00240726,  0.00621791,  0.01021052, ..., -0.01106928,
         0.00799661, -0.00016246]])

In [315]:
features = np.array(countvec2.get_feature_names())

In [356]:
arr= np.abs(logreg2.coef_)

In [357]:
indexes = arr.argsort()

In [362]:
indexes[0,-20:][::-1]

array([21639,     0, 21638,     1, 21637,     2,     3,     4,     5,
           6,     7, 21636, 21635,     8, 21634,     9,    10, 21633,
          11,    12], dtype=int64)

In [366]:
np.max(arr)

0.18139611783258958

In [364]:
features[indexes[0,-20:][::-1]]

array(['өөрөө', '00', 'өөрсдийгөө', '000', 'өөрийгөө', '0000',
       '000nana000', '00darya', '00okaay', '01', '02', 'өөр', 'өө', '03',
       'өнөөдөр', '04', '05', 'өмнө', '06', '07'],
      dtype='<U33')

In [303]:
feature_imp = np.vstack((np.array(countvec2.get_feature_names())[np.argsort(np.abs(logreg2.coef_))[::-1][:10]],
                        logreg2.coef_[0,np.argsort(np.abs(logreg2.coef_))][::-1][:10]))

In [308]:
feature_imp.T.shape

(21640, 2)

In [297]:
from  sklearn.metrics import confusion_matrix

In [298]:
data_cv_norm_ss_val = ss_cv_norm.transform(countvec2.transform(x_val.ttext_norm))

D:\distr\anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [306]:
prediction_cv_norm_ss_val= logreg.predict(data_cv_norm_ss_val)

In [307]:
confusion_matrix(prediction_cv_norm_ss_val, y_val)

array([[16181,  5645],
       [ 6299, 17242]])